In [2]:
from obspy import UTCDateTime
from obspy.clients.fdsn import Client
import matplotlib.pyplot as plt
from obspy.geodetics.base import gps2dist_azimuth,kilometer2degrees
from obspy.taup import TauPyModel
from obspy.signal.rotate import rotate_ne_rt,rotate_rt_ne
from matplotlib.ticker import MultipleLocator
from obspy import read, read_inventory, Stream, Trace, Catalog
import numpy as np
from obspy.io.sac.sacpz import attach_paz
from obspy.signal.filter import envelope
import pandas as pd
import os,glob
import warnings
warnings.filterwarnings("ignore")
model = TauPyModel(model="iasp91")
client = Client("IRIS")
plt.rcParams['font.sans-serif']='Times New Roman'

In [7]:
eventlist = ['201505240453','20140302201122','20150512211238']
cat = Catalog()
for event in eventlist:
    eve_time = UTCDateTime(event)
    eqevent = client.get_events(starttime=eve_time-5, endtime=eve_time+10*60,minmagnitude =6)
    cat+=eqevent

In [8]:
def checkday(num):
    if len(str(num)) == 1 :
        num = str(0)+str(num)
    return str(num)

def cal_lagtime(ref,cal_STA): 
    nx = len(ref)
    lags = np.arange(-nx+1 ,nx)
    ccov = np.correlate(ref - np.mean(ref), cal_STA-np.mean(cal_STA), mode = 'full')
    ccor = ccov / ((nx-1)* np.std(ref) * np.std(cal_STA))

    delta= 0.01
    ccv_X_Z = lags*delta
    ccv_Y_Z = ccor
    for i in range(len(lags)):
        if abs(ccv_Y_Z[i]) == max(abs(ccv_Y_Z)):
            ccv_y_Z = ccv_Y_Z[i]
            ccv_x_Z = lags[i]*delta
            lagtime = lags[i]
            lagtime_s = ccv_x_Z       
    print('lagtime=%s(s)' %(lagtime_s))
    return lagtime_s,max(abs(ccv_Y_Z))

In [22]:
network= ['TG','AM']
DATA_DIR = '/Volumes/home/Research/DataBase/00_'
PZ_DIR  =  '/Volumes/home/Research/DataBase/00_PZs'
INFO_DIR = '/Volumes/home/Research/DataBase/Armenia'
CSV_DIR = '/Volumes/home/Research/STEP/00_check_N'
freqmin=0.05#0.05
freqmax=1   #1 
#xmin=0; xmax = 26 #test 14s ##origin10  ##forS
plotxmin = 2      ##10
plotxmax = +10    ## +10 #for P ##+20 #forS
xmin=1 ; xmax = 4 ## for P
df = pd.read_csv(INFO_DIR+'/Station_info.csv')

#Badstation = ['DDFL','DGRG','LICH','LGD','NAVR','BATM','CANZ','BAUR','GANZ','BKRG','AZMN']
Badstation = ['LICH']
mmm= 5*10**-6
win = 'Pwindow' #'Swindow'

selectionSTA = 'ALAV'

In [23]:
for i in range(2,3):#len(cat)):
    eq_time = cat[i].origins[0].time
    print(eq_time)
    eq_lon = cat[i].origins[0].longitude
    eq_lat = cat[i].origins[0].latitude
    depth  = cat[i].origins[0].depth/1000
    mag    = cat[i].magnitudes[0].mag
    mag_type = cat[i].magnitudes[0].magnitude_type

#     try:
    inv1 = client.get_stations(network="GO", station='ONI', channel="HH?",
                                    starttime=eq_time,endtime=eq_time+30*60)

    ori_st = client.get_waveforms('GO','ONI','*','HH?',eq_time,eq_time+30*60,attach_response=True)
    print('GO ONI exist')
    ONI_st = ori_st.copy()
    ONI_st.remove_response(pre_filt =  [0.001,0.005,45,50], output="VEL")
    ONI_st.detrend('linear')
    ONI_st.detrend('demean')
    ONI_st.taper(0.05,type='cosine')


    st_lat = inv1[0][0].latitude
    st_lon = inv1[0][0].longitude
    dist,azi,baz = gps2dist_azimuth(eq_lat,eq_lon,st_lat,st_lon)
    deg = kilometer2degrees(dist/1000)
    Arr=model.get_travel_times(source_depth_in_km=depth,distance_in_degree=deg,phase_list=['P','S'])
    ONI_P_arr = Arr[0].time
    ONI_S_arr = Arr[1].time

    yyyy = eq_time.year
    mm = checkday(eq_time.month)
    dd = checkday(eq_time.day)
    hh = checkday(eq_time.hour)
    minn = checkday(eq_time.minute)
    
    deglist = [];STAlist=[]; BAZlist = []
    for net in network[1:2]:
        NET_DIR = f'{DATA_DIR}{net}'
        NET_PZs = f'{PZ_DIR}/{net}'
        eq_DIR =  f'{NET_DIR}/{yyyy}{mm}{dd}{hh}{minn}'
        if os.path.isdir(eq_DIR):
            figurebox = len(sorted(glob.glob(f'{eq_DIR}/*Z')))
            fig,ax = plt.subplots(nrows=figurebox , ncols=3,figsize=(15,15),sharex='col')
            for i, path in enumerate(sorted(glob.glob(f'{eq_DIR}/*Z'))):

                STA = path.rsplit('.',2)[1]

                if STA  not in Badstation:
                #if STA == selectionSTA :
                    ori_st = Stream()
                    st_lat = (df['lat'][ (df['network'] == net ) & (df['station'] == STA) ]).item()
                    st_lon = (df['lon'][ (df['network'] == net ) & (df['station'] == STA) ]).item()

                    dist,azi,baz = gps2dist_azimuth(eq_lat,eq_lon,st_lat,st_lon)
                    print(STA,baz )
                    dist = dist/1000
                    deg = kilometer2degrees(dist)
                    Arr=model.get_travel_times(source_depth_in_km=depth,distance_in_degree=deg,phase_list=['P','S'])
                    P_arr = Arr[0].time
                    S_arr = Arr[1].time
                    for datapath in glob.glob(f'{eq_DIR}/*{STA}.HH?'):
                            channel = datapath.rsplit('.',1)[-1]
                            tr4pz = Trace()
                            PZs = glob.glob(f'{NET_PZs}/{STA}/*{STA}_{channel}.txt')
                            if PZs == [] and net == 'TG' :
                                PZs = glob.glob(f'{NET_PZs}/ABST/*ABST_{channel}.txt')
                            elif PZs == [] and net == 'AM':
                                PZs = glob.glob(f'{NET_PZs}/ARZA/*ARZA_{channel}.txt')
                            attach_paz(tr4pz,PZs[0],tovel=True)
                            paz = dict(tr4pz.stats.paz)
                            tr = read(datapath,starttime=eq_time, endtime=eq_time+30*60)
                            tr.simulate(paz_remove=paz,pre_filt=(0.033, 0.034, 45, 50))
                            ori_st += tr

                    st = ori_st.copy()
                    st.merge(fill_value=0)
                    st.detrend('linear')
                    st.detrend('demean')
        #             st.taper(0.05,type='cosine')
                    #st.filter('bandpass',freqmin=freqmin,freqmax=freqmax,corners=4,zerophase=True)
                    
                    for win in ['Pwindow','Swindow']:
                        if win == 'Pwindow': xmin=0 ; xmax = 6 ; freqmin = 0.1; freqmax = 1 ;plotxmin = 2;plotxmax=10 ; title = f'P b.p. {freqmin}-{freqmax}Hz window {xmin}~{xmax}s\n'
                        elif win == 'Swindow': xmin=-5; xmax = 20; freqmin = 0.05; freqmax = 0.2;plotxmin = 30; plotxmax=20; title = f'{title}S-b.p. {freqmin}-{freqmax} Hz window {xmin}~{xmax}s'
                        
                        if win == 'Pwindow':
                            ONI_filter =ONI_st.copy()
                            ONI_filter.filter('bandpass',freqmin=freqmin,freqmax=freqmax,corners=4,zerophase=True)
                            ONI_times = ONI_st.select(component='E')[0].times(reftime=eq_time+ONI_P_arr)
                            xmin_ind = np.where(ONI_times==min(ONI_times, key=lambda x:abs(x-(xmin))))[0][0]
                            xmax_ind = np.where(ONI_times==min(ONI_times, key=lambda x:abs(x-(xmax))))[0][0]
                        elif win == 'Swindow':
                            ONI_filter =ONI_st.copy()
                            ONI_filter.filter('bandpass',freqmin=freqmin,freqmax=freqmax,corners=4,zerophase=True)
                            ONI_times = ONI_st.select(component='E')[0].times(reftime=eq_time+ONI_S_arr)
                            xmin_ind = np.where(ONI_times==min(ONI_times, key=lambda x:abs(x-(xmin))))[0][0]
                            xmax_ind = np.where(ONI_times==min(ONI_times, key=lambda x:abs(x-(xmax))))[0][0]
                       
                        ONI_HHE = ONI_filter.select(component='E')[0].data
                        ONI_HHN = ONI_filter.select(component='N')[0].data
                        ONI_HHZ = ONI_filter.select(component='Z')[0].data
                        ONI_HHR,ONI_HHT = rotate_ne_rt(ONI_HHN,ONI_HHE,baz)
                        ONI_R_window = ONI_HHR[xmin_ind:xmax_ind]
                        ONI_T_window = ONI_HHT[xmin_ind:xmax_ind]
                        ONI_Z_window = ONI_HHZ[xmin_ind:xmax_ind]
                       
                        if win == 'Pwindow':
                            filter_st =st.copy()
                            filter_st.filter('bandpass',freqmin=freqmin,freqmax=freqmax,corners=4,zerophase=True)
                            times = st[0].times(reftime=eq_time+P_arr)
                            xmin_ind = np.where(times==min(times, key=lambda x:abs(x-(xmin))))[0][0]
                            xmax_ind = np.where(times==min(times, key=lambda x:abs(x-(xmax))))[0][0]
                        elif win == 'Swindow':
                            filter_st =st.copy()
                            filter_st.filter('bandpass',freqmin=freqmin,freqmax=freqmax,corners=4,zerophase=True)
                            times = st[0].times(reftime=eq_time+S_arr)
                            xmin_ind = np.where(times==min(times, key=lambda x:abs(x-(xmin))))[0][0]
                            xmax_ind = np.where(times==min(times, key=lambda x:abs(x-(xmax))))[0][0]
                        
                        HHE = filter_st.select(component='E')[0].data
                        HHN = filter_st.select(component='N')[0].data
                        HHZ = filter_st.select(component='Z')[0].data
                        ori_HHR , ori_HHT    = rotate_ne_rt(HHN,HHE,baz) 
                        ori_R_window = ori_HHR[xmin_ind:xmax_ind]
                        ori_T_window = ori_HHT[xmin_ind:xmax_ind]
                        ori_Z_window = HHZ[xmin_ind:xmax_ind]
                        
                        
                        if win == 'Pwindow': 
                            j = 0
                            lagtime, coeff = cal_lagtime(ONI_R_window,ori_R_window)
                            if abs(lagtime) >=4.0: lagtime = 0.0
                               
                            ax[i,j].plot(times+lagtime,ori_HHR,'k')
                            ax[i,j].plot(ONI_times,ONI_HHR,'r')
                            ax[i,j].set_ylim(-6*10**-6,6*10**-6)
#                             ax[i,j].set_ylim(-6*10**-6,*10**-6)
                            ymin,ymax=ax[i,j].get_ylim()
                            ax[i,j].text(xmin-plotxmin+1,ymin,f'{round(coeff,2)}, {round(lagtime,2)}',color='k',fontsize=10,ha = 'left', va ='bottom')
                            ax[i,j].set_xlim(xmin-plotxmin,xmax+plotxmax)
                            ax[i,j].set_ylabel(STA,fontsize=15)
                        elif win == 'Swindow': 
                            ymin =-2.5*10**-6; ymax = -ymin
                            j = 1
                            ax[i,j].vlines(x = 0, ymin=ymin,ymax = ymax , color = 'grey', linestyle ='--')
                            lagtime, coeff = cal_lagtime(ONI_R_window,ori_R_window)
                            if abs(lagtime) >=4.0: lagtime = 0.0
                            ax[i,j].plot(times+lagtime,ori_HHR,'k')
                            ax[i,j].plot(ONI_times,ONI_HHR,'r')
                            #ax[i,j].set_ylim(-4*10**-6,4*10**-6)
                            ax[i,j].set_ylim(ymin,ymax)
                            ymin,ymax=ax[i,j].get_ylim()
                            ax[i,j].text(xmin-plotxmin+1,ymin,f'{round(coeff,2)}, {round(lagtime,2)}',color='k',fontsize=10,ha = 'left', va ='bottom')
                            ax[i,j].set_xlim(xmin-plotxmin,xmax+plotxmax)
                            
                            j = 2 
                            lagtimeT, coeff = cal_lagtime(ONI_T_window,ori_T_window)
                            ax[i,j].vlines(x = 0, ymin = ymin,ymax = ymax , color = 'grey', linestyle ='--')
                            ax[i,j].plot(times+lagtime,ori_HHT,'k')
                            ax[i,j].plot(ONI_times,ONI_HHT,'r')
                            #ax[i,j].set_ylim(-4*10**-6,4*10**-6)
                            ax[i,j].set_ylim(ymin,ymax)
                            ymin,ymax=ax[i,j].get_ylim()
                            ax[i,j].text(xmin-plotxmin+1,ymin,f'{round(coeff,2)}, {round(lagtime,2)}',color='k',fontsize=10,ha = 'left', va ='bottom')
                            ax[i,j].set_xlim(xmin-plotxmin,xmax+plotxmax)
                            
                            
                            
        ax[0,0].set_title('P (R comp.)', fontsize=16 ,fontweight = 'bold')                    
        ax[0,1].set_title('S (R comp.)', fontsize=16 ,fontweight = 'bold')                      
        ax[0,2].set_title('S (T comp.)', fontsize=16 ,fontweight = 'bold')   
        ax[i,0].set_xlabel('Ori. of P arr.', fontsize=15) 
        ax[i,1].set_xlabel('Ori. of S arr.', fontsize=15) 
        ax[i,2].set_xlabel('Ori. of S arr.', fontsize=15) 
        plt.suptitle(f'{yyyy}{mm}{dd}{hh}{minn} {net} compared with ONI\nLat:{eq_lat} Lon:{eq_lon} Depth: {depth}km Mw: {mag} Dist~{round(deg,1)}\n{title}',fontsize=20, fontweight = 'bold')  
        plt.savefig(f'{CSV_DIR}/{yyyy}{mm}{dd}{hh}{minn}_{net}_originRT4PS.png',dpi=150,facecolor='white')
        plt.close()
                            

2015-05-12T21:12:58.600000Z
GO ONI exist
ARZA 54.85252207002687
lagtime=0.94(s)
lagtime=0.64(s)
lagtime=0.69(s)
BAUR 54.633433761755136
lagtime=0.44(s)
lagtime=6.12(s)
lagtime=-3.76(s)
BYUR 54.64612024558873
lagtime=0.31(s)
lagtime=0.74(s)
lagtime=0.3(s)
GERK 55.05321555491441
lagtime=0.59(s)
lagtime=0.3(s)
lagtime=0.61(s)
KECH 55.17920869343031
lagtime=1.29(s)
lagtime=-6.39(s)
lagtime=1.91(s)
MAGY 54.91205979522846
lagtime=1.53(s)
lagtime=1.89(s)
lagtime=2.49(s)
SHEN 54.738332158438034
lagtime=0.99(s)
lagtime=0.6(s)
lagtime=1.51(s)
VAND 54.359239580235304
lagtime=1.66(s)
lagtime=-3.79(s)
lagtime=-0.56(s)
ZARN 54.447124408261644
lagtime=-0.3(s)
lagtime=-7.85(s)
lagtime=4.02(s)


In [75]:
np.where(ONI_times==min(ONI_times, key=lambda x:abs(x-(xmin))))[0][0]

121743

In [94]:
qqq=client.get_waveforms('GO','AKH','*','HH?',eq_time,eq_time+30*60,attach_response=True)

In [98]:
qqq[0].stats

               network: GO
               station: AKH
              location: 
               channel: HHE
             starttime: 2014-03-02T20:11:22.710000Z
               endtime: 2014-03-02T20:41:22.710000Z
         sampling_rate: 100.0
                 delta: 0.01
                  npts: 180001
                 calib: 1.0
_fdsnws_dataselect_url: http://service.iris.edu/fdsnws/dataselect/1/query
               _format: MSEED
                 mseed: AttribDict({'dataquality': 'M', 'number_of_records': 677, 'encoding': 'STEIM2', 'byteorder': '>', 'record_length': 512, 'filesize': 994304})
            processing: ['ObsPy 1.2.2: trim(endtime=UTCDateTime(2014, 3, 2, 20, 41, 22, 710000)::fill_value=None::nearest_sample=True::pad=False::starttime=UTCDateTime(2014, 3, 2, 20, 11, 22, 710000))']
              response: Channel Response
	From M/S (Velocity in Meters Per Second) to COUNTS (Digital Counts)
	Overall Sensitivity: 9.84864e+08 defined at 1.000 Hz
	7 stages:
		Stage 1: PolesZerosRe